In [ ]:
from heapq import heappush, heappop
import pandas as pd
from multiprocessing import Pool

ROWS, COLS = 5, 5
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def extract_locs_from_csv(file_name):
    df = pd.read_csv(file_name)
    return df

def is_valid_move(grid, x, y):
    return 0 <= x < ROWS and 0 <= y < COLS and grid[x][y] == 0

def manhattan_distance(loc, goal):
    return abs(loc[0] - goal[0]) + abs(loc[1] - goal[1])

def heuristic(grid, goals):
    distance = 0
    for i in range(ROWS):
        for j in range(COLS):
            if grid[i][j] != 0:
                distance += manhattan_distance((i, j), goals[grid[i][j] - 1])
    return distance

def astar(start_grid, goal_grid, goal_locs):
    start_state = tuple(map(tuple, start_grid))
    goal_state = tuple(map(tuple, goal_grid))

    open_list = [(heuristic(start_grid, goal_locs), start_grid, 0, [get_locs_from_grid(start_grid)])]
    visited = set()

    while open_list:
        _, current_grid, cost, path = heappop(open_list)

        current_tuple_grid = tuple(map(tuple, current_grid))
        if current_tuple_grid == goal_state:
            return path

        for i in range(ROWS):
            for j in range(COLS):
                if current_grid[i][j] != 0:
                    for dx, dy in directions:
                        ni, nj = i + dx, j + dy
                        if is_valid_move(current_grid, ni, nj):
                            new_grid = [list(row) for row in current_grid]
                            new_grid[ni][nj], new_grid[i][j] = new_grid[i][j], 0
                            str_new_grid = str(new_grid)
                            if str_new_grid not in visited:
                                h = heuristic(new_grid, goal_locs)
                                new_path = list(path)
                                new_path.append(get_locs_from_grid(new_grid))
                                heappush(open_list, (cost + h + 1, new_grid, cost + 1, new_path))
                                visited.add(str_new_grid)
    return None

def get_locs_from_grid(grid):
    locs = []
    for i in range(ROWS):
        for j in range(COLS):
            if grid[i][j] != 0:
                locs.append((i, j))
    return locs

def make_grid_from_locs(locs):
    grid = [[0] * COLS for _ in range(ROWS)]
    for i, (x, y) in enumerate(locs):
        grid[x][y] = i + 1
    return grid

def solve_problem(row):
    start_loc = eval(row['start_loc'])
    goal_loc = eval(row['goal_loc'])
    print(f'[problem id {row.name}] start locations: {start_loc}')
    print(f'[problem id {row.name}] goal locations: {goal_loc}')
    return astar(make_grid_from_locs(start_loc), make_grid_from_locs(goal_loc), goal_loc)

def main(file_name):
    assignment = extract_locs_from_csv(file_name)

    # Use multiprocessing to speed up the A* algorithm
    with Pool() as pool:
        Solutions = pool.map(solve_problem, assignment.iterrows())

    codes = []
    for solution in Solutions:
        code = []
        for i in range(len(solution) - 1):
            for r in range(len(solution[0])):
                if solution[i][r] != solution[i + 1][r]:
                    code += [int(f'{r}{solution[i + 1][r][0]}{solution[i + 1][r][1]}')]
        codes += [code]

    submission = pd.DataFrame(codes)
    submission['id'] = assignment.index
    submission = submission.set_index('id')
    submission = submission.fillna('-')
    submission.to_csv('submission.csv')
    print(submission)

file_name = 'assignment1.csv'
main(file_name)


In [ ]:
from heapq import heappush, heappop
import pandas as pd
from concurrent.futures import ProcessPoolExecutor

ROWS, COLS = 5, 5
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def extract_locs_from_csv(file_name):
    df = pd.read_csv(file_name)
    return df

def is_valid_move(grid, x, y):
    return 0 <= x < ROWS and 0 <= y < COLS and grid[x][y] == 0

def manhattan_distance(loc, goal):
    return abs(loc[0] - goal[0]) + abs(loc[1] - goal[1])

def heuristic(grid, goals):
    distance = 0
    for i in range(ROWS):
        for j in range(COLS):
            if grid[i][j] != 0:
                distance += manhattan_distance((i, j), goals[grid[i][j] - 1])
    return distance

def astar(start_grid, goal_grid, goal_locs):
    start_state = tuple(map(tuple, start_grid))
    goal_state = tuple(map(tuple, goal_grid))

    open_list = [(heuristic(start_grid, goal_locs), start_grid, 0, [get_locs_from_grid(start_grid)])]
    visited = set()

    while open_list:
        _, current_grid, cost, path = heappop(open_list)

        current_tuple_grid = tuple(map(tuple, current_grid))
        if current_tuple_grid == goal_state:
            return path

        for i in range(ROWS):
            for j in range(COLS):
                if current_grid[i][j] != 0:
                    for dx, dy in directions:
                        ni, nj = i + dx, j + dy
                        if is_valid_move(current_grid, ni, nj):
                            new_grid = [list(row) for row in current_grid]
                            new_grid[ni][nj], new_grid[i][j] = new_grid[i][j], 0
                            str_new_grid = str(new_grid)
                            if str_new_grid not in visited:
                                h = heuristic(new_grid, goal_locs)
                                new_path = list(path)
                                new_path.append(get_locs_from_grid(new_grid))
                                heappush(open_list, (cost + h + 1, new_grid, cost + 1, new_path))
                                visited.add(str_new_grid)
    return None

def get_locs_from_grid(grid):
    locs = []
    for i in range(ROWS):
        for j in range(COLS):
            if grid[i][j] != 0:
                locs.append((i, j))
    return locs

def make_grid_from_locs(locs):
    grid = [[0] * COLS for _ in range(ROWS)]
    for i, (x, y) in enumerate(locs):
        grid[x][y] = i + 1
    return grid

def solve_problem(row):
    start_loc = eval(row['start_loc'])
    goal_loc = eval(row['goal_loc'])
    print(f'[problem id {row.name}] start locations: {start_loc}')
    print(f'[problem id {row.name}] goal locations: {goal_loc}')
    return astar(make_grid_from_locs(start_loc), make_grid_from_locs(goal_loc), goal_loc)

def main(file_name):
    assignment = extract_locs_from_csv(file_name)

    # Use ProcessPoolExecutor to execute solve_problem function in parallel
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(solve_problem, row) for _, row in assignment.iterrows()]
    
        Solutions = []
        for future in futures:
            solution = future.result()
            Solutions.append(solution)

    codes = []
    for solution in Solutions:
        code = []
        for i in range(len(solution) - 1):
            for r in range(len(solution[0])):
                if solution[i][r] != solution[i + 1][r]:
                    code += [int(f'{r}{solution[i + 1][r][0]}{solution[i + 1][r][1]}')]
        codes += [code]

    submission = pd.DataFrame(codes)
    submission['id'] = assignment.index
    submission = submission.set_index('id')
    submission = submission.fillna('-')
    submission.to_csv('submission.csv')
    print(submission)

file_name = 'assignment1.csv'
main(file_name)


In [ ]:
from collections import deque
import csv

SIZE = 5
MOVES = [(1, 0), (-1, 0), (0, 1), (0, -1)]

def bfs(board, start, goal):
    visited = set()
    queue = deque([(start, [])])
    
    while queue:
        (x, y), path = queue.popleft()
        
        if (x, y) == goal:
            return path + [goal]
        
        for dx, dy in MOVES:
            nx, ny = x + dx, y + dy
            if 0 <= nx < SIZE and 0 <= ny < SIZE and board[nx][ny] == 0 and (nx, ny) not in visited:
                visited.add((nx, ny))
                queue.append(((nx, ny), path + [(nx, ny)]))
    return []

def move_all_pieces(start_loc, goal_loc):
    board = [[0 for _ in range(SIZE)] for _ in range(SIZE)]
    for s in start_loc:
        board[s[0]][s[1]] = 1

    pairs = list(zip(start_loc, goal_loc))
    pairs.sort(key=lambda x: len(bfs(board, x[0], x[1])) if bfs(board, x[0], x[1]) else float('inf'))

    sequence_of_moves = []

    for start, goal in pairs:
        path = bfs(board, start, goal)
        if path:
            for pos in path[1:]:
                board[start[0]][start[1]] = 0
                board[pos[0]][pos[1]] = 1
                sequence_of_moves.append(list(start_loc))
                start_loc.remove(start)
                start_loc.append(pos)
                start = pos  # Update start for next iteration
    return sequence_of_moves

def process_csv(filename):
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            _, start, goal, _ = row
            start_loc = eval(start)
            goal_loc = eval(goal)
            
            solutions = move_all_pieces(start_loc, goal_loc)
            
            for solution in solutions:
                print(solution)

filename = "assignment1.csv"
process_csv(filename)


In [ ]:
from collections import deque
import csv

# Extracting start_locs and goal_locs from the CSV
def extract_locs_from_csv(file_name):
    with open(file_name, 'r') as file:
        reader = csv.reader(file)
        start_locs, goal_locs = [], []
        for row in reader:
            start, goal = row[0], row[1]
            start_x, start_y = map(int, start.split(','))
            goal_x, goal_y = map(int, goal.split(','))
            start_locs.append((start_x, start_y))
            goal_locs.append((goal_x, goal_y))
    return start_locs, goal_locs

# BFS and helper functions from previous code snippet go here...

def main(file_name):
    start_locs, goal_locs = extract_locs_from_csv(file_name)
    
    moves = least_moves_to_goal(start_locs, goal_locs)
    
    if moves == -1:
        print("There is no solution.")
    else:
        print(f"The least number of moves required: {moves}")

# Call the function
file_name = 'assignment1.csv'
main(file_name)

ROWS, COLS = 5, 5
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # right, down, left, up

def is_valid_move(grid, x, y):
    return 0 <= x < ROWS and 0 <= y < COLS and grid[x][y] == 0

def bfs(start_config, goal_config):
    start_state = tuple(start_config)
    goal_state = tuple(goal_config)
    queue = deque([(start_state, 0)])
    visited = set()
    visited.add(start_state)

    while queue:
        current_config, steps = queue.popleft()

        if current_config == goal_state:
            return steps

        for i, (x, y) in enumerate(current_config):
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if is_valid_move(grid, nx, ny):
                    new_config = list(current_config)
                    new_config[i] = (nx, ny)
                    new_config.sort()  # Sorting to ensure uniqueness of state representation
                    new_tuple_config = tuple(new_config)
                    if new_tuple_config not in visited:
                        queue.append((new_tuple_config, steps + 1))
                        visited.add(new_tuple_config)
    return -1  # No valid path exists

def least_moves_to_goal(start_locs, goal_locs):
    # Initializing the grid
    grid = [[0] * COLS for _ in range(ROWS)]
    for loc in start_locs:
        x, y = loc
        grid[x][y] = 1

    moves = bfs(start_locs, goal_locs)
    return moves


print(least_moves_to_goal(start_locs, goal_locs))


In [ ]:
from collections import deque
import csv

# Extracting start_locs and goal_locs from the CSV
def extract_locs_from_csv(file_name):
    with open(file_name, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skipping the header row
        start_locs, goal_locs = [], []
        for row in reader:
            start, goal = row[0], row[1]
            try:
                start_x, start_y = map(int, start.split(','))
                goal_x, goal_y = map(int, goal.split(','))
                start_locs.append((start_x, start_y))
                goal_locs.append((goal_x, goal_y))
            except ValueError:
                # Ignoring rows that can't be converted to integer pairs
                pass
    return start_locs, goal_locs

ROWS, COLS = 5, 5
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # right, down, left, up

def is_valid_move(grid, x, y, current_config):
    return 0 <= x < ROWS and 0 <= y < COLS and grid[x][y] == 0 and (x, y) not in current_config

def bfs(start_config, goal_config):
    start_state = tuple(start_config)
    goal_state = tuple(goal_config)
    queue = deque([(start_state, 0)])
    visited = set()
    visited.add(start_state)

    while queue:
        current_config, steps = queue.popleft()
        # Reset the grid for the current configuration
        grid = [[0] * COLS for _ in range(ROWS)]
        for x, y in current_config:
            grid[x][y] = 1

        if current_config == goal_state:
            return steps

        for i, (x, y) in enumerate(current_config):
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if is_valid_move(grid, nx, ny, current_config):
                    new_config = list(current_config)
                    new_config[i] = (nx, ny)
                    new_config.sort()  # Sorting to ensure uniqueness of state representation
                    new_tuple_config = tuple(new_config)
                    if new_tuple_config not in visited:
                        queue.append((new_tuple_config, steps + 1))
                        visited.add(new_tuple_config)
    return -1  # No valid path exists

def least_moves_to_goal(start_locs, goal_locs):
    moves = bfs(start_locs, goal_locs)
    return moves

def main(file_name):
    start_locs, goal_locs = extract_locs_from_csv(file_name)
    moves = least_moves_to_goal(start_locs, goal_locs)
    print(start_locs)
    print(goal_locs)

    if moves == -1:
        print("There is no solution.")
    else:
        print(f"The least number of moves required: {moves}")

# Call the function
file_name = 'assignment1.csv'
main(file_name)



In [1]:
import csv
import ast
from heapq import heappush, heappop

ROWS, COLS = 5, 5
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def extract_locs_from_csv(file_name):
    with open(file_name, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        start_locs, goal_locs = [], []
        for row in reader:
            start, goal = ast.literal_eval(row[1]), ast.literal_eval(row[2])
            start_locs.append(start)
            goal_locs.append(goal)
    return start_locs, goal_locs

def is_valid_move(grid, x, y):
    return 0 <= x < ROWS and 0 <= y < COLS and grid[x][y] == 0

def manhattan_distance(loc, goal):
    return abs(loc[0] - goal[0]) + abs(loc[1] - goal[1])

def heuristic(grid, goals):
    distance = 0
    for i in range(ROWS):
        for j in range(COLS):
            if grid[i][j] != 0:
                distance += manhattan_distance((i, j), goals[grid[i][j] - 1])
    return distance

def astar(start_grid, goal_grid, goal_locs):
    start_state = tuple(map(tuple, start_grid))
    goal_state = tuple(map(tuple, goal_grid))

    open_list = [(heuristic(start_grid, goal_locs), start_grid, 0)]
    visited = set()

    while open_list:
        _, current_grid, cost = heappop(open_list)
        
        if tuple(map(tuple, current_grid)) == goal_state:
            return cost

        for i in range(ROWS):
            for j in range(COLS):
                if current_grid[i][j] != 0:
                    for dx, dy in directions:
                        ni, nj = i + dx, j + dy
                        if is_valid_move(current_grid, ni, nj):
                            new_grid = [list(row) for row in current_grid]
                            new_grid[ni][nj], new_grid[i][j] = new_grid[i][j], 0
                            if tuple(map(tuple, new_grid)) not in visited:
                                h = heuristic(new_grid, goal_locs)
                                heappush(open_list, (cost + h + 1, new_grid, cost + 1))
                                visited.add(tuple(map(tuple, new_grid)))
    return -1

def make_grid_from_locs(locs):
    grid = [[0] * COLS for _ in range(ROWS)]
    for i, (x, y) in enumerate(locs):
        grid[x][y] = i + 1
    return grid

def main(file_name):
    start_lists, goal_lists = extract_locs_from_csv(file_name)
    
    for start_locs, goal_locs in zip(start_lists, goal_lists):
        start_grid = make_grid_from_locs(start_locs)
        goal_grid = make_grid_from_locs(goal_locs)

        moves = astar(start_grid, goal_grid, goal_locs)
        
        if moves == -1:
            print(f"Start: {start_locs}, Goal: {goal_locs} - There is no solution.")
        else:
            print(f"Start: {start_locs}, Goal: {goal_locs} - Minimum moves: {moves}")

file_name = 'assignment1.csv'
main(file_name)


Start: [(4, 4), (2, 3), (4, 1), (0, 3), (1, 0), (1, 2), (1, 4)], Goal: [(1, 1), (1, 3), (2, 2), (3, 3), (0, 1), (1, 4), (3, 1)] - Minimum moves: 24
Start: [(2, 1), (2, 3), (3, 2), (1, 1), (0, 3), (0, 0), (1, 3)], Goal: [(4, 0), (2, 3), (2, 1), (1, 1), (4, 3), (2, 4), (1, 3)] - Minimum moves: 17
Start: [(0, 0), (3, 0), (1, 4), (2, 0), (3, 2), (1, 3), (3, 4)], Goal: [(4, 3), (2, 3), (1, 1), (1, 0), (1, 3), (0, 2), (0, 4)] - Minimum moves: 23
Start: [(4, 1), (3, 4), (0, 1), (2, 2), (0, 4), (1, 1), (3, 0)], Goal: [(0, 0), (3, 3), (1, 3), (3, 2), (1, 0), (0, 2), (1, 1)] - Minimum moves: 20
Start: [(3, 2), (4, 0), (2, 2), (1, 0), (3, 0), (2, 0), (3, 1)], Goal: [(0, 2), (1, 2), (0, 1), (2, 1), (3, 3), (4, 1), (2, 4)] - Minimum moves: 23
Start: [(3, 2), (1, 3), (1, 4), (2, 1), (3, 3), (2, 0), (0, 3)], Goal: [(2, 3), (3, 3), (4, 0), (4, 3), (3, 2), (3, 4), (4, 4)] - Minimum moves: 26
Start: [(0, 4), (1, 4), (3, 2), (4, 1), (1, 3), (4, 2), (0, 0)], Goal: [(1, 4), (1, 0), (0, 4), (4, 1), (1, 1), 

In [1]:
from heapq import heappush, heappop
import pandas as pd
from multiprocessing import Pool

ROWS, COLS = 5, 5
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def extract_locs_from_csv(file_name):
    df = pd.read_csv(file_name)
    return df

def is_valid_move(grid, x, y):
    return 0 <= x < ROWS and 0 <= y < COLS and grid[x][y] == 0

def manhattan_distance(loc, goal):
    return abs(loc[0] - goal[0]) + abs(loc[1] - goal[1])

def heuristic(grid, goals):
    distance = 0
    for i in range(ROWS):
        for j in range(COLS):
            if grid[i][j] != 0:
                distance += manhattan_distance((i, j), goals[grid[i][j] - 1])
    return distance

def astar(start_grid, goal_grid, goal_locs):
    start_state = tuple(map(tuple, start_grid))
    goal_state = tuple(map(tuple, goal_grid))

    open_list = [(heuristic(start_grid, goal_locs), start_grid, 0, [get_locs_from_grid(start_grid)])]
    visited = set()

    while open_list:
        _, current_grid, cost, path = heappop(open_list)

        current_tuple_grid = tuple(map(tuple, current_grid))
        if current_tuple_grid == goal_state:
            return path

        for i in range(ROWS):
            for j in range(COLS):
                if current_grid[i][j] != 0:
                    for dx, dy in directions:
                        ni, nj = i + dx, j + dy
                        if is_valid_move(current_grid, ni, nj):
                            new_grid = [list(row) for row in current_grid]
                            new_grid[ni][nj], new_grid[i][j] = new_grid[i][j], 0
                            str_new_grid = str(new_grid)
                            if str_new_grid not in visited:
                                h = heuristic(new_grid, goal_locs)
                                new_path = list(path)
                                new_path.append(get_locs_from_grid(new_grid))
                                heappush(open_list, (cost + h + 1, new_grid, cost + 1, new_path))
                                visited.add(str_new_grid)
    return None

def get_locs_from_grid(grid):
    locs = []
    for i in range(ROWS):
        for j in range(COLS):
            if grid[i][j] != 0:
                locs.append((i, j))
    return locs

def make_grid_from_locs(locs):
    grid = [[0] * COLS for _ in range(ROWS)]
    for i, (x, y) in enumerate(locs):
        grid[x][y] = i + 1
    return grid

def main(file_name):
    assignment = extract_locs_from_csv(file_name)
    Solutions = []
    for i, row in assignment.iterrows():
        start_loc = eval(row['start_loc'])
        goal_loc = eval(row['goal_loc'])
        print(f'[problem id {i}] start locations: {start_loc}')
        print(f'[problem id {i}] goal locations: {goal_loc}')
        Solutions.append(astar(make_grid_from_locs(start_loc), make_grid_from_locs(goal_loc), goal_loc))
        
    codes = []
    for solution in Solutions:
        code = []
        for i in range(len(solution) - 1):
            for r in range(len(solution[0])):
                if solution[i][r] != solution[i + 1][r]:
                    code += [int(f'{r}{solution[i + 1][r][0]}{solution[i + 1][r][1]}')]
        codes += [code]
    submission = pd.DataFrame(codes)
    submission['id'] = assignment.index
    submission = submission.set_index('id')
    submission = submission.fillna('-')
    submission.to_csv('submission.csv')
    print(submission)

file_name = 'assignment1.csv'
main(file_name)
